## Финальный проект на тему "Чат-бот для перевода с иностранного языка"
### Выполнила: студентка БМБ2204 Валерия Локтева

Телеграмм бот был написан с использованием:
1) Библиотеки aiogram для реализации самого бота
2) Библиотеки googletrans для реализации перевода
3) Библиотеки sqlite3 для реализации базы данных
4) Модуля nest_asyncio для исправления несовместимости asyncio c Jupiter Notebook (А именно файлами .ipynb)

### ВАЖНО: Бот переводит исключительно С иностранного, по умолчанию стоит китайский язык, язык можно сменить

Критерии:
1. Практическая применимость - Применяется для перевода и составления сборников переводов администратором.

2. Понятный интерфейс - Имеется множество инлайн кнопок для различных функций, а также при запросе данных объясняется что требуют.

3. Условия - Применяется в проверках на администратора:
  ```
      if(message.from_user.id == 673023617 or message.from_user.id == 1645913880):
        await bot.send_message(message.chat.id, text, reply_markup= keyboard_adm)
      else:
        await bot.send_message(message.chat.id, text, reply_markup= keyboard)
   А также в проверках на присутствие в базе данных.
  ```

4. Цикл - Цикл for используется в функции create_inline_kb для создания инлайн клавиатуры. А также в методе create_phrasebook_inline_kb для создания инлайн клавиатуры из динамических данных (из данных, взятых из базы)
    ```
    def create_phrasebook_inline_kb(width: int):
    phrasebooks = using_database.get_themes()
    kb_builder = InlineKeyboardBuilder()
    buttons: list[InlineKeyboardButton] = []
    **for key, value in phrasebooks.items():
        buttons.append(InlineKeyboardButton(
            text= str(value),
            callback_data= PhraseCallbackFactory(
                phrasebook_id = key,
                phrasebook_theme = value
                ).pack()
        )
        )
    buttons.append(InlineKeyboardButton(text='Главное меню', callback_data='main_menu'))
    kb_builder.row(*buttons, width=width)
    return kb_builder.as_markup()
    ```
5. Словари и множества:
       Используется словарь BUTTONS с парами ключ-значение, для создания клавиатуры инлайн кнопок, ключом является callback.data , а в значении лежит текст кнопок.

      В качестве множества у нас список кнопок в функциях create_inline_kb и create_phrasebook_inline_kb, мы создаем список кнопок для того, чтобы потом с помощью билдера сделать клавиатуру.

6. Запись в файл: В качестве записи в файл у нас используется запись в базу данных с помощью sqlite3, а именно в базу данных my_database.db, с таблицами:  
        1  helping_phrase   - Для отслеживания и хранения информации о разговорнике, который просматривает админ.  
        2  phrasebook - Для хранения и использования информации о разговорниках (id, тема, контент)  
        3  user - Для хранения и использования информации о пользователе (tg_id , username, язык который он поставил в настройках перевода)  

7. Комментарии в коде: Присутствуют комментарии для каждого обработчика апдейтов и функций.

8. Правильность кода на любом запуске : Программа должна запускаться правильно. 

9. Своя функция: Проект буквально состоит из своих функций которые вызываются при срабатывании обработчиков.

10. Новая функция: Библиотеки aiogram, sqlite3, модуль googletrans. (В последней ячейке будет написано подробнее
)

In [ ]:
!pip install nest_asyncio
!pip install aiogram==3.3.0
!pip install googletrans==4.0.0rc1

In [2]:
# Импортируем все необходимые модули и библиотеки
from aiogram import Bot, Dispatcher, F
from aiogram.filters import Command, CommandStart, StateFilter
from aiogram.filters.callback_data import CallbackData
from aiogram.types import ( CallbackQuery, InlineKeyboardButton,
                            InlineKeyboardMarkup, Message)
from aiogram.fsm.state import State, StatesGroup
from aiogram.fsm.context import FSMContext
from aiogram.fsm.storage.memory import MemoryStorage
from aiogram.utils.keyboard import InlineKeyboardBuilder
from googletrans import Translator
import nest_asyncio
import sqlite3

# Сохраняем токен нашего бота для работы.
BOT_TOKEN = '6520049981:AAF-yNLl_mRceny3IrP_6DEfeqyfZmouBTc'

#Это нужно чтобы ipynb не ругался на asyncio.
nest_asyncio.apply()
storage = MemoryStorage()

#Инициализируем объект бота и диспетчера.
bot = Bot(BOT_TOKEN)
dp = Dispatcher()

#Функция нужна для создания базы данных с таблицей helping_phrase.
# Ничего не принимает и не возвращает.
def creating_helping_phrase():
    connection = sqlite3.connect('my_database.db')
    cursor = connection.cursor()
    # Создаем таблицу helping_phrase
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS helping_phrase(
               admin_id INTEGER,
               phrasebook_id INTEGER
               )
    ''')
    # Сохраняем изменения и закрываем соединение
    connection.commit()
    connection.close()

#Функция нужна для создания таблицы Phrasebook
# Ничего не принимает и не возвращает.
def creating_phrasebook():
    connection = sqlite3.connect('my_database.db')
    cursor = connection.cursor()
    # Создаем таблицу Phrasebook
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS phrasebook (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                theme TEXT,
                content TEXT
                )
    ''')
    # Сохраняем изменения и закрываем соединение
    connection.commit()
    connection.close()

#Функция нужна для создания таблицы User
# Ничего не принимает и не возвращает.
def creating_user():
    connection = sqlite3.connect('my_database.db')
    cursor = connection.cursor()
    # Создаем таблицу User
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS  user (
                 user_id INTEGER,
                 username TEXT,
                 language TEXT
               )
    ''')
    # Сохраняем изменения и закрываем соединение
    connection.commit()
    connection.close()

creating_helping_phrase()
creating_phrasebook()
creating_user()

#Фабрика callback'ов для динамических данных из базы данных.
class PhraseCallbackFactory(CallbackData, prefix='ph'):
    phrasebook_id: int
    phrasebook_theme: str

#Тут у нас лежат различные состояния для отслеживания состояния пользователя
class FSMFillForm(StatesGroup):
    client_login = State()
    waiting_for_text = State()
    waiting_for_theme = State()
    waiting_for_choice = State()
    waiting_for_ru = State()
    waiting_for_ino = State()

#Этот хэндлер обабатывает команду /start и выводит в ответ главное меню с кнопками
@dp.message(CommandStart())
async def process_start_command(message: Message, state: FSMContext):
    #Очищаем состояние на всякий случай
    await state.clear()
    #Создаем клавиатуры быстро и просто благодаря функции (идея взята из курса по телеграмм ботам)
    keyboard = create_inline_kb(1, 'but_1','but_2','but_3')
    keyboard_adm =  create_inline_kb(1, 'but_1','but_2','but_3', 'but_4_adm')
    text = 'Привет! Этот бот может сделать перевод для тебя или показать слова для диалога. Что бы ты хотел перевести? \nДля запуска переводчика нажми соответствующую кнопку.\n По умолчанию бот переводит с китайского, язык можно поменять через соответствующую кнопку.'
    new_user(message.from_user.id, message.from_user.username)
    #Проверка на админа
    if(message.from_user.id == 673023617 or message.from_user.id == 1645913880):
        await bot.send_message(message.chat.id, text, reply_markup= keyboard_adm)

    else:
        await bot.send_message(message.chat.id, text, reply_markup= keyboard)

#Этот хэндлер срабатывает на нажатие кнопки "Главное меню" и выводит его
@dp.callback_query(F.data == 'main_menu')
async def main_menu(callback: CallbackQuery, state: FSMContext):
    await bot.delete_message(callback.message.chat.id, callback.message.message_id)
    #Очищаем состояние на всякий случай
    await state.clear()
    #Создаем клавиатуры быстро и просто благодаря функции (идея взята из курса по телеграмм ботам)
    keyboard = create_inline_kb(1, 'but_1','but_2','but_3')
    keyboard_adm =  create_inline_kb(1, 'but_1','but_2','but_3', 'but_4_adm')
    text = 'Привет! Этот бот может сделать перевод для тебя или показать слова для диалога. Что бы ты хотел перевести? \nДля запуска переводчика нажми соответствующую кнопку.\n По умолчанию бот переводит с китайского, язык можно поменять через соответствующую кнопку.'
    #Проверка на админа
    if(callback.from_user.id == 673023617 or callback.from_user.id == 1645913880):
        await bot.send_message(callback.message.chat.id, text, reply_markup= keyboard_adm)

    else:
        await bot.send_message(callback.message.chat.id, text, reply_markup= keyboard)

#Этот хэндлер срабатывает на кнопку "Запустить переводчик" и врубает ожидание сообщений от пользователя
@dp.callback_query(F.data == 'but_1')
async def start_translate(callback: CallbackQuery, state: FSMContext):
    await bot.delete_message(callback.message.chat.id, callback.message.message_id)
    keyboard = create_inline_kb(1, 'main_menu')
    await bot.send_message(callback.message.chat.id, 'Отправьте пожалуйста сообщение, с тектом, который надо перевести', reply_markup= keyboard)
    #Врубаем ожидание
    await state.set_state(FSMFillForm.waiting_for_text)
# Этот хэндлер ловит сообщение при условии состояния ожидания сообщения
@dp.message(StateFilter(FSMFillForm.waiting_for_text))
async def translate(message: Message, state: FSMContext):
    lang = get_lang(message.from_user.id)
    translated = translatе_text(message.text, 'ru', lang)
    keyboard = create_inline_kb(1, 'main_menu')
    result = translated + '\n\nМожете дальше отправлять фразы\слова для перевода!'
    await bot.send_message(message.chat.id, result, reply_markup= keyboard)
    #Врубаем ожидание
    await state.set_state(FSMFillForm.waiting_for_text)

# Этот хэндлер нужен для того, чтобы отлавливать нажатие на "Изменить язык" и последующей смены языка для пользователя
@dp.callback_query(F.data == 'but_2')
async def change_language(callback: CallbackQuery):
    await bot.delete_message(callback.message.chat.id, callback.message.message_id)
    keyboard = create_inline_kb(1, 'zh-cn', 'en', 'fr','de','cs', 'main_menu')
    await bot.send_message(callback.message.chat.id, 'Выберите пожалуйста язык перевода', reply_markup= keyboard)

# Меняет язык на китайский
@dp.callback_query(F.data == 'zh-cn')
async def china(callback: CallbackQuery):
    await bot.delete_message(callback.message.chat.id, callback.message.message_id)
    keyboard = create_inline_kb(1, 'main_menu')
    change_lang( callback.from_user.id,'zh-cn')
    await bot.send_message(callback.message.chat.id, 'Отлично! Язык успешно изменен, вы можете вернуться в главное меню для дальнейшего использования бота', reply_markup= keyboard)
# Меняет язык на английский
@dp.callback_query(F.data == 'en')
async def english(callback: CallbackQuery):
    await bot.delete_message(callback.message.chat.id, callback.message.message_id)
    keyboard = create_inline_kb(1, 'main_menu')
    change_lang(callback.from_user.id, 'en')
    await bot.send_message(callback.message.chat.id, 'Отлично! Язык успешно изменен, вы можете вернуться в главное меню для дальнейшего использования бота', reply_markup= keyboard)
# Меняет язык на французский
@dp.callback_query(F.data == 'fr')
async def french(callback: CallbackQuery):
    await bot.delete_message(callback.message.chat.id, callback.message.message_id)
    keyboard = create_inline_kb(1, 'main_menu')
    change_lang(callback.from_user.id,'fr')
    await bot.send_message(callback.message.chat.id, 'Отлично! Язык успешно изменен, вы можете вернуться в главное меню для дальнейшего использования бота', reply_markup= keyboard)
# Меняет язык на немецкий
@dp.callback_query(F.data == 'de')
async def germany(callback: CallbackQuery):
    await bot.delete_message(callback.message.chat.id, callback.message.message_id)
    keyboard = create_inline_kb(1, 'main_menu')
    change_lang(callback.from_user.id,'de')
    await bot.send_message(callback.message.chat.id, 'Отлично! Язык успешно изменен, вы можете вернуться в главное меню для дальнейшего использования бота', reply_markup= keyboard)
# Меняет язык на чешский
@dp.callback_query(F.data == 'cs')
async def czesh(callback: CallbackQuery):
    await bot.delete_message(callback.message.chat.id, callback.message.message_id)
    keyboard = create_inline_kb(1, 'main_menu')
    change_lang(callback.from_user.id,'cs')
    await bot.send_message(callback.message.chat.id, 'Отлично! Язык успешно изменен, вы можете вернуться в главное меню для дальнейшего использования бота', reply_markup= keyboard)

# Этот хэндлер срабатывает на кнопку "Разговорники" и выводит сообщение с кнопками с текстом - темами разговорников
@dp.callback_query(F.data == 'but_3')
async def show_phrasebooks(callback: CallbackQuery, state: FSMContext):
    await bot.delete_message(callback.message.chat.id, callback.message.message_id)
    keyboard = create_phrasebook_inline_kb(1)
    await bot.send_message(callback.message.chat.id, 'Выберите пожалуйста разговорник, чтобы получить информацию о нем:', reply_markup= keyboard)

# Этот хэндлер срабатывает на нажатие на кнопку разговорника и выводит всю информацию о нем, а также позволяет админам добавлять в него записи и удалять разговорник.
@dp.callback_query(PhraseCallbackFactory.filter())
async def choice_phrasebook(callback: CallbackQuery,
                                 callback_data: PhraseCallbackFactory):
    await bot.delete_message(callback.message.chat.id, callback.message.message_id)
    content = str(get_content(callback_data.phrasebook_id))
    if(content == "None"):
        content = "Разговорник пока пуст!"
    theme = str(callback_data.phrasebook_theme)
    phrasebook_id = callback_data.phrasebook_id
    user_id = callback.from_user.id
    if(callback.from_user.id == 673023617 or callback.from_user.id == 1645913880):
        if (checking_phrase(user_id)):
            this_phrasebook_set(user_id,phrasebook_id)
        else:
            this_phrasebook_new_set(user_id,phrasebook_id)
        keyboard = create_inline_kb(1,'but_5', 'but_6', 'main_menu')
    else:
        keyboard = create_inline_kb(1, 'main_menu')

    result = f'Вот информация по разговорнику {theme}' + '\n'+ content
    await bot.send_message(callback.message.chat.id, result, reply_markup= keyboard)
# Хэндлер срабатывает на кнопку добавления информации в разговорник, Врубает ожидание русского текста
@dp.callback_query(F.data == 'but_5')
async def new_text(callback: CallbackQuery, state: FSMContext):
    await bot.delete_message(callback.message.chat.id, callback.message.message_id)
    keyboard = create_inline_kb(1, 'main_menu')
    await bot.send_message(callback.message.chat.id, 'Отлично! Если вы видите это сообщение - вы админ) Введите пожалуйста на русском текст записи', reply_markup= keyboard)
    await state.set_state(FSMFillForm.waiting_for_ru)
# Этот хэндлер ловит сообщение при включенном ожидании русского текста и добавляет его в разговорник, а потом ждет иностранный перевод
@dp.message(StateFilter(FSMFillForm.waiting_for_ru))
async def ru_content(message: Message, state: FSMContext):
    user_id = message.from_user.id
    page = this_phrasebook_get(user_id)
    update_content_ru(message.text, page)
    await bot.send_message(message.chat.id, 'А теперь отправьте перевод нужный вам')
    await state.set_state(FSMFillForm.waiting_for_ino)
# Этот хэндлер  ловит сообщение при включенном ожидании иностранного перевода записывает его.
@dp.message(StateFilter(FSMFillForm.waiting_for_ino))
async def ino_content(message: Message, state: FSMContext):
    user_id = message.from_user.id
    page = this_phrasebook_get(user_id)
    update_content_ino(message.text, page)
    keyboard = create_inline_kb(1, 'main_menu')
    await bot.send_message(message.chat.id, 'Новая запись была сделана!', reply_markup= keyboard)
    await state.clear()
# Этот хэндлер срабатывает на кнопку удаления разговорника и удаляет его.
@dp.callback_query(F.data == 'but_6')
async def delete_phrasebook(callback: CallbackQuery, state: FSMContext):
    await bot.delete_message(callback.message.chat.id, callback.message.message_id)
    user_id = callback.from_user.id
    page = this_phrasebook_get(user_id)
    delete_phrasebook(page)
    keyboard = create_inline_kb(1, 'main_menu')
    await bot.send_message(callback.message.chat.id, 'Разговорник был успешно удален', reply_markup= keyboard)
# Этот хэндлер срабатывает на кнопку добавления нового разговорника и при незаполненности хранилища разговорников позволяет создать новый, врубая ожидание темы.
@dp.callback_query(F.data == 'but_4_adm')
async def create_phrasebook(callback: CallbackQuery, state: FSMContext):
    await bot.delete_message(callback.message.chat.id, callback.message.message_id)
    keyboard = create_inline_kb(1, 'main_menu')
    count = count_phrasebook()
    if (count >= 10):
        text = 'Извините( Но количество ваших разговорников максимально (10)'
    else:
        text = 'Хорошо) Давайте создадим новый разговорник!\n\n\n\nОтправьте пожалуйста название вашего разговорника'
        await state.set_state(FSMFillForm.waiting_for_theme)
    await bot.send_message(callback.message.chat.id, text, reply_markup= keyboard)

# Этот хэндлер срабатывает на сообщение при включенном ожидании темы.
@dp.message(StateFilter(FSMFillForm.waiting_for_theme))
async def creating_phrasebook(message: Message, state: FSMContext):
    new_phrasebook(message.text)
    keyboard = create_inline_kb(1, 'main_menu')
    await bot.send_message(message.chat.id, 'Новый разговорник создан! Можете вернуться в главное меню)', reply_markup= keyboard)

#Словарь с фиксированной информацией о кнопках (текст и callback_data)
BUTTONS: dict[str, str] = {
    'but_1': 'Запустить переводчик',
    'but_2': 'Изменить язык перевода',
    'but_3': 'Разговорники',
    'but_4_adm': 'Добавить разговорник',
    'main_menu': 'Главное меню',
    'zh-cn': 'Китайский',
    'en': 'Английский',
    'fr': 'Французский',
    'de': 'Немецкий',
    'cs': 'Чешский',
    'but_5': 'Новая запись в Разговорник',
    'but_6': 'Удалить разговорник',
    }


#Функция для быстрого создания инлайн клавиатуры.
# Принимает в себя число width - количество кнопок в ряду, *args - несколько аргументов в виде строк, или **kwargs - Несколько пар в виде строк.
# Возвращает клавиатуру.
def create_inline_kb(width: int,
                     *args: str,
                     **kwargs: str) -> InlineKeyboardMarkup:
    # Инициализируем билдер
    kb_builder = InlineKeyboardBuilder()
    # Инициализируем список для кнопок
    buttons: list[InlineKeyboardButton] = []

    # Заполняем список кнопками из аргументов args и kwargs
    if args:
        for button in args:
            buttons.append(InlineKeyboardButton(
                text=BUTTONS[button] if button in BUTTONS else button,
                callback_data=button))
    if kwargs:
        for button, text in kwargs.items():
            buttons.append(InlineKeyboardButton(
                text=text,
                callback_data=button))

    # Распаковываем список с кнопками в билдер методом row c параметром width
    kb_builder.row(*buttons, width=width)

    # Возвращаем объект инлайн-клавиатуры
    return kb_builder.as_markup()

#Функция для перевода текста из русского в иностранный
# Принимает в себя текст, который надо перевести и языки с и на которые надо перевести.
# Возвращает перевод.
def translatе_text(text, to_lang, from_lang):
    translator = Translator()
    translation = translator.translate(text, dest=to_lang, src = from_lang)
    return translation.text
#Проверка языка на нужный
# Принимает в себя текст, который надо проверить на то, на каком он языке и язык, который должен быть.
# Возвращает результат проверки.
def check_language(text, from_lang):
    translator = Translator()
    det = translator.detect(text)
    if (det.lang == from_lang):
        return True
    else:
        return False
#Функция для создания клавиатуры из базы данных, использует фабрику callback'ов
#Принимает в себя число width - количество кнопок в ряду.
# Возвращает клавиатуру с нужными кнопками.
def create_phrasebook_inline_kb(width: int):
    phrasebooks = get_themes()
    kb_builder = InlineKeyboardBuilder()
    buttons: list[InlineKeyboardButton] = []
    # Создаем кнопки из пар (ключ-значение словаря с данными о разговорниках)
    for key, value in phrasebooks.items():
        buttons.append(InlineKeyboardButton(
            text= str(value),
            # Создаем callback_data с помощью фабрики коллбэков
            callback_data= PhraseCallbackFactory(
                phrasebook_id = key,
                phrasebook_theme = value
                ).pack()
        )
        )
    #Добавляем кнопку главного меню
    buttons.append(InlineKeyboardButton(text='Главное меню', callback_data='main_menu'))
    kb_builder.row(*buttons, width=width)
    return kb_builder.as_markup()

#Функция записи информации о пользователе в базу
# Принимает в себя айди и логин пользователя.
# Ничего не возвращает.
def new_user(id, username):
    connection = sqlite3.connect('my_database.db')
    cursor = connection.cursor()
    if(checking_user):
         cursor.execute(f"INSERT INTO user (user_id, username, language) VALUES({id},'{str(username)}', 'zh-cn')")
    connection.commit()
    connection.close()
# Функция проверки наличия такого человека в базе по его id
# Принимает в себя айди пользователя
# Возвращает результат проверки.
def checking_user(id):
    connection = sqlite3.connect('my_database.db')
    cursor = connection.cursor()
    cursor.execute(f"SELECT user_id FROM user WHERE user_id = {id}")
    result = cursor.fetchone()[0]
    connection.close()
    if (result):
        return False
    else:
        return True

# Меняет язык пользователя в базе
# Принимает в себя айди пользователя и новый язык.
# Ничего не возвращает
def change_lang(id, lang):
    connection = sqlite3.connect('my_database.db')
    cursor = connection.cursor()
    cursor.execute(f"UPDATE user SET language = '{lang}' WHERE user_id = {id}")
    connection.commit()
    connection.close()

#Получает информацию о языке пользователя с базы
# Принимает в себя айди пользователя
# Возвращает язык, который выбран у пользователя.
def get_lang(id):
    connection = sqlite3.connect('my_database.db')
    cursor = connection.cursor()
    cursor.execute(f"SELECT language FROM user WHERE user_id = {id}")
    result = cursor.fetchone()[0]
    connection.commit()
    connection.close()
    return result
# Получает список кортежей с двумя значениями из базы данных и переводит его в словарь.
# Ничего не принимает.
# Возвращает словарь с парами айди-тема из базы данных разговорников.
def get_themes():
    connection = sqlite3.connect('my_database.db')
    cursor = connection.cursor()
    cursor.execute("SELECT id, theme FROM phrasebook")
    result = cursor.fetchall()
    connection.commit()
    connection.close()

    diction = {key: value for key, value in result}
    return diction
# Создает новый разговорник
# Принимает тема разговорника.
# Ничего не возвращает.
def new_phrasebook(theme):
    connection = sqlite3.connect('my_database.db')
    cursor = connection.cursor()
    cursor.execute(f"INSERT INTO phrasebook (theme, content) VALUES('{theme}', '')")
    connection.commit()
    connection.close()
# Получает информацию о содержании разговорника
# Принимает в себя айди разговорника.
# Возвращает всю информацию о разговорнике.
def get_content(id):
    connection = sqlite3.connect('my_database.db')
    cursor = connection.cursor()
    cursor.execute(f"SELECT content FROM phrasebook WHERE id = {id}")
    result = cursor.fetchone()[0]
    connection.commit()
    connection.close()
    return result
#Получает тему разговорника по id
# Принимает в себя айди разговорника.
# Возвращает тему этого разговорника.
def phrasebook_theme(id):
    connection = sqlite3.connect('my_database.db')
    cursor = connection.cursor()
    cursor.execute(f"SELECT theme FROM phrasebook WHERE id = {id}")
    result = cursor.fetchone()[0]
    connection.commit()
    connection.close()
    return result
# Обновляет содержание разговорника, добавляя русскую часть из text
# Принимает в себя дополнительный текст для разговорника и айди разговорника.
# Ничего не возвращает
def update_content_ru(text, id):
    connection = sqlite3.connect('my_database.db')
    cursor = connection.cursor()
    new_content = get_content(id)
    new_content = new_content +  "\n" + text
    cursor.execute(f"UPDATE phrasebook SET content = '{new_content}' WHERE id = {id}")
    connection.commit()
    connection.close()
# Обновляет содержание разговорника, добавляя иностранную часть из text.
# Принимает в себя дополнительный текст для разговорника и айди разговорника.
# Ничего не возвращает.
def update_content_ino(text, id):
    connection = sqlite3.connect('my_database.db')
    cursor = connection.cursor()
    new_content = get_content(id)
    new_content = new_content + "—" + text
    cursor.execute(f"UPDATE phrasebook SET content = '{new_content}' WHERE id = {id}")
    connection.commit()
    connection.close()
# Создает запись в таблицу helping_phrase для хранения информации о текущем разговорнике,  просматриваемым пользователем.
# Принимает в себя айди администрации и айди разговорника.
# Ничего не возвращает.
def this_phrasebook_new_set(admin_id, ph_id):
    connection = sqlite3.connect('my_database.db')
    cursor = connection.cursor()
    cursor.execute(f"INSERT INTO helping_phrase (admin_id, phrasebook_id) VALUES({admin_id}, {ph_id})")
    connection.commit()
    connection.close()
# Обновляет информацию о текущем разговорнике,  просматриваемым пользователем.
# Принимает в себя айди администрации и айди разговорника.
# Ничего не возвращает.
def this_phrasebook_set(admin_id, ph_id):
    connection = sqlite3.connect('my_database.db')
    cursor = connection.cursor()
    cursor.execute(f"UPDATE helping_phrase SET phrasebook_id = {ph_id} WHERE admin_id = {admin_id}")
    connection.commit()
    connection.close()
#Проверяет существование пользователя в helping_phrase
# Принимает в себя айди администрации.
# Возвращает результат проверки айди.
def checking_phrase(admin_id):
    connection = sqlite3.connect('my_database.db')
    cursor = connection.cursor()
    cursor.execute(f"SELECT admin_id FROM helping_phrase WHERE admin_id = {admin_id}")
    result = cursor.fetchone()
    connection.close()
    if (result):
        return True
    else:
        return False
# Получет айди разговорника, который сейчас смотрит пользователь.
# Принимает в себя айди администрации.
# Возвращает айди разговорника, который сейчас просматривается.
def this_phrasebook_get(admin_id):
    connection = sqlite3.connect('my_database.db')
    cursor = connection.cursor()
    cursor.execute(f"SELECT phrasebook_id FROM helping_phrase WHERE admin_id = {admin_id}")
    result = cursor.fetchone()[0]
    connection.close()
    return result
#Удаляет разговорник из базы данных.
# Принимает в себя айди разговорника.
# Ничего не возвращает.
def delete_phrasebook(id):
    connection = sqlite3.connect('my_database.db')
    cursor = connection.cursor()
    cursor.execute(f"DELETE FROM phrasebook WHERE id = {id}")
    connection.commit()
    connection.close()
# Получает информацию о количестве разговорников
# Ничего не принимает.
# Возвращает количество разговорников в базе.
def count_phrasebook():
    connection = sqlite3.connect('my_database.db')
    cursor = connection.cursor()
    cursor.execute(f"SELECT COUNT(*) FROM phrasebook")
    result = cursor.fetchone()[0]
    connection.close()
    return result

# Запускаем поллинг (Постоянное опрашивание сервера Телеграмм на наличие новых обновлений)
if __name__ == '__main__':
    dp.run_polling(bot)

Failed to fetch updates - TelegramNetworkError: HTTP Client says - ClientOSError: [Errno 1] [SSL: APPLICATION_DATA_AFTER_CLOSE_NOTIFY] application data after close notify (_ssl.c:2706)
Sleep for 1.000000 seconds and try again... (tryings = 0, bot id = 6520049981)


## Использованные библиотеки

### Асинхронная библиотека aiogram, для Telegram Bot API, нужна собственно для самой реализации бота. Импортируем из нее самого бота, диспетчер для обработки хэндлеров и много много много всего. Документация: https://docs.aiogram.dev/en/latest/

### Библиотека sqlite3, нужна для реализации базы данных для хранения и работы с информацией. Документация:https://www.sqlite.org/docs.html

### Модуль nest_asyncio для исправления ошибок ipynb и asyncio. Описание: https://pypi.org/project/nest-asyncio/

### Библиотека Googletrans с реализацией API Google Translate, использовала для перевода сообщений. Документация: https://py-googletrans.readthedocs.io/en/latest/